# Evaluation

In [ ]:
from bib_dedupe.dedupe_benchmark import DedupeBenchmarker
from bib_dedupe.bib_dedupe import BibDeduper
from bib_dedupe.util import BibDedupeUtil
from asreview.data import load_data, ASReviewData
from datetime import datetime
import pandas as pd

In [ ]:
bd_util = BibDedupeUtil()
merge_updated_papers = True

# for benchmark_path in reversed(bd_util.get_dataset_labels()):
for benchmark_path in bd_util.get_dataset_labels():
    if benchmark_path in ["problem_cases"]: # "problem_cases" digital_work "srsr", "depression"
         continue
    print(f"Dataset: {benchmark_path}")
    
    dedupe_benchmark = DedupeBenchmarker(benchmark_path=f"../data/{benchmark_path}", merge_updated_papers=merge_updated_papers)
    records_df = dedupe_benchmark.get_records_for_dedupe()
    
    # Bib-dedupe
    dedupe_instance = BibDeduper()
    timestamp = datetime.now()
    actual_blocked_df = dedupe_instance.block(records_df=records_df)
    matches = dedupe_instance.match(actual_blocked_df, merge_updated_papers=merge_updated_papers)
    merged_df = dedupe_instance.merge(records_df, matches=matches)
    result = dedupe_benchmark.compare_dedupe_id(records_df=records_df, merged_df=merged_df, timestamp=timestamp)
    bd_util.append_to_output(result, package_name="bib-dedupe")

    # More detailed comparison for debugging
    dedupe_benchmark.export_cases(prepared_records_df=records_df, blocked_df=actual_blocked_df, matches=matches)
    
    # ASReview
    asdata = ASReviewData(records_df)
    timestamp = datetime.now()
    merged_df = asdata.drop_duplicates()
    result = dedupe_benchmark.compare_dedupe_id(records_df=records_df, merged_df=merged_df, timestamp=timestamp)
    bd_util.append_to_output(result, package_name="asreview")
    print()
    